In [313]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import time
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/controller_acapy_and_tests/jupyter_util.py'>

In [314]:
localhost = "http://localhost:"

def assert_status_code_is_200(response_object):
    get = response_object
    if get.status_code != 200:
        print("status_code: " + str(get.status_code) + "  value: " + str(get.text))
        assert get.status_code == 200, "status_code is not 200"

def get(port, url_end) -> dict:
    port = str(port)
    request = requests.get(localhost + port + url_end)
    assert_status_code_is_200(request)
    result = json.loads(request.text)
    return result
    
def post(port, url_end, json_dict=None) -> dict:
    port = str(port)
    if json_dict != None:
        request = requests.post(localhost + port + url_end, json=json_dict)
    else:
        request = requests.post(localhost + port + url_end)
        
    assert_status_code_is_200(request)
    result = json.loads(request.text)
    return result    

In [315]:
def save_to_pds(some_json):
    add = requests.post('http://localhost:8150/pds/save', json=some_json)
    assert_status_code_is_200(add)
    
    add = json.loads(add.text)
    return add

def save_to_pds2(some_json):
    add = requests.post('http://localhost:8151/pds/save', json=some_json)
    assert_status_code_is_200(add)
    
    add = json.loads(add.text)
    return add

def get_from_pds(some_id):
    get = requests.get('http://localhost:8150/pds/' + some_id)
    assert_status_code_is_200(get)    
        
    get = json.loads(get.text)
    return get

def get_from_pds2(some_id):
    get = requests.get('http://localhost:8150/pds/' + some_id)
    assert_status_code_is_200(get)    
        
    get = json.loads(get.text)
    return get

def save_and_retrieve(optional_test_string = ""):
    payload = {"payload": "testing" + " " + optional_test_string}
    id = save_to_pds(payload)
    content = get_from_pds(str(id['payload_id']))
    
    assert payload == content, f"initial payload not equal to return value from pds {payload, content}"
    
    return id['payload_id']

def get_pdses():
    get1 = requests.get('http://localhost:8150/pds')
    get2 = requests.get('http://localhost:8151/pds')
    
    assert get1.text != None and get2.text != None
    assert_status_code_is_200(get1)
    assert_status_code_is_200(get2)
    
    return [get1.text, get2.text]
        
        
def set_pds_config(pds_type, settings = {}, instance_name = "default"):
    if settings != {}:
        settings.update({"optional_instance_name": instance_name})
        config = requests.post('http://localhost:8150/pds/settings', json={
          "settings": {
              pds_type: settings
          }
        })
        assert_status_code_is_200(config)    
        
    config = requests.post('http://localhost:8150/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config)   
    
    if settings != {}:
        settings.update({"optional_instance_name": instance_name})
        config = requests.post('http://localhost:8151/pds/settings', json={
          "settings": {
              pds_type: settings
          }
        })
        assert_status_code_is_200(config)    
        
    config = requests.post('http://localhost:8151/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config) 
    
    
def set_own_your_data():
    set_pds_config("own_your_data", {
        "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
        "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
    })
    
def activate_pds_both_agents(pds_type, instance_name = 'default'):
    config = requests.post('http://localhost:8150/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config)   
    config = requests.post('http://localhost:8151/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config) 
    
def test_pds_return_same_id():
    set_own_your_data()
    id1 = save_to_pds({"payload": "testing"})
    set_pds_config("local")
    id2 = save_to_pds({"payload": "testing"})
    
    assert id1['payload_id'] == id2['payload_id'], "own_your_data and local return different ids"
    
    set_pds_config("local")
    id1 = save_to_pds({"payload": "testing"})
    set_pds_config("data_vault", {"host": "http://ocadatavault"})
    id2 = save_to_pds({"payload": "testing"})
    
    assert id1['payload_id'] == id2['payload_id'], "local and data_vault return different ids"

    return [id1, id2]
    
    
# test_pds_return_same_id()
set_pds_config("local")
save_and_retrieve("local")
set_pds_config("data_vault", {"host": "http://ocadatavault"})
save_and_retrieve("data_vault_thcf")
set_pds_config("data_vault", {"host": "http://ocadatavault"}, "vault2")
save_and_retrieve("data_vault_thcf_2")
# set_own_your_data()
# payload_id = save_and_retrieve("own_your_data")
get_pdses()



['{"active": "data_vault, vault2", "types": ["local", "data_vault", "own_your_data"]}',
 '{"active": "data_vault, vault2", "types": ["local", "data_vault", "own_your_data"]}']

In [316]:
## TEST that Settings are persistent! 
# activate_pds_both_agents("data_vault")
# save_and_retrieve("data_vault_thcf")
# activate_pds_both_agents("data_vault", "vault2")
# save_and_retrieve("data_vault_thcf_2")
# activate_pds_both_agents("own_your_data")
# payload_id = save_and_retrieve("own_your_data")

In [319]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)

agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

Decoded invitation:

{'@id': 'a11111ad-37d8-4fe9-9dce-9ae10f3089ae',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['51Vn23AqepDruN2K6vrmoWcxfBhG2SF1bNx6GGUKmYwy'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [325]:
# time.sleep(0.2)

# add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={1234}')
# print(add.text)

# set_pds_config("own_your_data", {
#     "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
#     "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
# })
# payload_id = save_and_retrieve("own_your_data1234")
# add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={payload_id}')
# assert '{"message_sent": "success"}' == add.text

# time.sleep(1)
# get1 = requests.get('http://localhost:8150/pds/' + str(payload_id))
# get2 = requests.get('http://localhost:8151/pds/' + str(payload_id))
# print(get1.text, get2.text)
# assert get1.status_code == get2.status_code, f"status codes are not equal {get1.status_code, get2.status_code}"
# assert get1.text == get2.text, "exchanged data with get_from is not equal"
# set_pds_config("data_vault", {"host": "http://ocadatavault"})

### FEATURE DISCOVERY

In [326]:
data_dri = save_to_pds2({"payload": "testing"})
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "data_dri": data_dri['payload_id'],
  }
})
print(add.text)
add = json.loads(add.text)

{"service_schema": {"oca_schema_namespace": "string", "oca_schema_dri": "string"}, "updated_at": "2020-11-16 14:05:26.328827Z", "label": "consent_test", "created_at": "2020-11-16 14:05:26.328827Z", "consent_schema": {"oca_schema_namespace": "consent", "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", "data_dri": "zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR"}}


In [327]:
time.sleep(0.5)
newDid1 = util.createDid(agents[1])
newDid0 = util.createDid(agents[0])

try:
    util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])
except:
    pass
try:
    util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])
except:
    pass

util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])
time.sleep(0.5)

In [348]:
issue_cred = post(8150, '/issue-credential/request', {
  "credential_values": {"test": "stuff"},
  "connection_id": connection[0],
  "credential_type": "string"
})

pp.pprint(issue_cred)

{'credential_exchange_id': 'f8a66496-466b-4f3e-b4d8-a6c5f155d85b'}


In [349]:
cred_exchange = get(8151, "/issue-credential/exchange/record?state=request_received")
print(cred_exchange)

[{'state': 'request_received', 'thread_id': 'fb2b323f-e2e6-4d03-a266-638d7a0ef398', 'credential_request': {'credential_type': 'string', 'credential_values': {'test': 'stuff'}}, 'trace': False, 'updated_at': '2020-11-16 14:05:41.575586Z', 'role': 'issuer', 'created_at': '2020-11-16 14:05:41.575586Z', 'connection_id': 'ddff1481-6a1e-4df5-9baa-77e43d2251ee', 'credential_exchange_id': '8c8ca6ea-b9ce-4240-a76e-3dad7ca81c51', 'initiator': 'external'}, {'state': 'request_received', 'thread_id': '2a859f38-afb5-4d39-87d8-483d1c06c9da', 'credential_request': {'credential_type': 'string', 'credential_values': {'test': 'stuff'}}, 'trace': False, 'updated_at': '2020-11-16 13:53:20.218293Z', 'role': 'issuer', 'created_at': '2020-11-16 13:53:20.218293Z', 'connection_id': '00afdfac-7deb-4d78-a09f-b3816ef74fcd', 'credential_exchange_id': 'a0df94a9-c689-4994-bf26-481d243c6f00', 'initiator': 'external'}, {'state': 'request_received', 'thread_id': '91077a80-2e9d-4e40-a0e8-5d15d92fa903', 'credential_reques

In [350]:
id = cred_exchange[0]['credential_exchange_id']
issue_cred = post(8151, "/issue-credential/send?credential_exchange_id=" + id)

pp.pprint(issue_cred)

('{"context": ["https://www.w3.org/2018/credentials/v1", '
 '"https://www.schema.org"], "type": ["VerifiableCredential", "string"], '
 '"issuer": "XSYpKEcRL2TZ6vrAP6cafG", "issuanceDate": "2020-11-16 '
 '14:14:59.329169Z", "credentialSubject": {"test": "stuff", "id": '
 '"RcEexRd8f9nLVhXqWnqNDK"}, "proof": {"type": "Ed25519Signature2018", '
 '"created": "2020-11-16 14:14:59.345261Z", "proofPurpose": "assertionMethod", '
 '"verificationMethod": "4E2p6AZW2d11veBHdGeqd9dE9QKWJRRaFLykHf2chvdt", "jws": '
 '"ec14e5e78610b064e3f5879bde1dbdf6db6373935b74e5564c06430fa98d3b499de33a14dcfe9dc014a4edff941cedcdd47d3d584c2ca5e1bbf2dd7424590303"}}')


In [351]:
issue_cred = post(8151, '/present-proof/request', {
  "context": [
    "string"
  ],
  "connection_id": connection[1],
  "requested_attributes": {
    "test": {
      "restrictions": [
        {}
      ]
    }
  },
  "type": [
    "string"
  ]
})

pp.pprint(issue_cred)

{'test': {'restrictions': [{}]}}


In [360]:
exchange_record = get(8151, "/present-proof/exchange/record?state=request_received")
print(exchange_record)

[{'presentation_request': {'type': ['string'], 'context': ['string'], 'requested_attributes': {'test': {'restrictions': [{}]}}, 'nonce': '5702ecbd-3191-46a0-aaa2-f4d22b271312'}, 'state': 'request_received', 'auto_present': False, 'trace': False, 'updated_at': '2020-11-16 14:07:58.063868Z', 'role': 'prover', 'presentation_exchange_id': '2ce1c99d-84d4-4103-b339-35fca7a7dded', 'created_at': '2020-11-16 14:07:58.063868Z', 'connection_id': 'ddff1481-6a1e-4df5-9baa-77e43d2251ee', 'thread_id': '2be8c44b-1e74-4592-89ca-a7c839069a75', 'initiator': 'external'}]


In [353]:
cred_exchange = get(8150, "/credentials")
print(cred_exchange)

{'results': [{'created_at': '2020-11-16 14:14:59.400809Z', 'updated_at': '2020-11-16 14:14:59.400809Z', 'type': ['VerifiableCredential', 'string'], 'issuer': 'XSYpKEcRL2TZ6vrAP6cafG', 'proof': {'type': 'Ed25519Signature2018', 'created': '2020-11-16 14:14:59.345261Z', 'proofPurpose': 'assertionMethod', 'verificationMethod': '4E2p6AZW2d11veBHdGeqd9dE9QKWJRRaFLykHf2chvdt', 'jws': 'ec14e5e78610b064e3f5879bde1dbdf6db6373935b74e5564c06430fa98d3b499de33a14dcfe9dc014a4edff941cedcdd47d3d584c2ca5e1bbf2dd7424590303'}, 'credentialSubject': {'test': 'stuff', 'id': 'RcEexRd8f9nLVhXqWnqNDK'}, 'issuanceDate': '2020-11-16 14:14:59.329169Z', 'id': '67b65058-e4b5-422f-9e78-418216770778'}, {'created_at': '2020-11-16 14:06:31.406701Z', 'updated_at': '2020-11-16 14:06:31.406701Z', 'type': ['VerifiableCredential', 'string'], 'issuer': 'PixogHbwtsd6JmGgXoCDTR', 'proof': {'type': 'Ed25519Signature2018', 'created': '2020-11-16 14:06:31.367086Z', 'proofPurpose': 'assertionMethod', 'verificationMethod': '7j9GyDyY

In [364]:
issue_cred = post(8151, '/present-proof/present', {
  "requested_credentials": {
    "test": {
        "cred_id": cred_exchange['results'][1]['id']
    }
  },
  "exchange_record_id": exchange_record[0]['presentation_exchange_id']
})

pp.pprint(issue_cred)

status_code: 500  value: 500: credential_id f0569e55-119e-4b19-9cba-58913bd51f2b for field test is invalid.


AssertionError: status_code is not 200

In [20]:
cred = json.loads(issue_cred.text)
pp.pprint(cred)

{'@context': ['https://www.w3.org/2018/credentials/v1',
              'https://www.schema.org'],
 'credentialSubject': {'id': 'LZPRK7bURBE22Jz59atbLv', 'test': 'stuff'},
 'issuanceDate': '2020-11-06 12:24:07.646624Z',
 'issuer': 'VEjkV7bi6G5iM54DNJysJf',
 'proof': {'created': '2020-11-06 12:24:07.665158Z',
           'jws': '7cc9797473a4edcf19784f05b710ffa2eaf2aa2819dbce33bc26045fb4b18f395b480db6bc0a048f2e0ae78f7d2b30596b773a7b2eeb874e8ee3caa6ace8050d',
           'proofPurpose': 'assertionMethod',
           'type': 'Ed25519Signature2018',
           'verificationMethod': 'CeW2Ga6u76nDPyTxxvLdbWruAyG3mPM4Lc3tLaeCUzY9'},
 'type': ['VerifiableCredential', 'string']}


In [36]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
request_service_list = requests.get('http://localhost:8150/verifiable-services/request-service-list/' + connection[0])
print(request_service_list.text)

print(service_list)
service_list = json.loads(service_list.text)
pp.pprint(service_list)

"SUCCESS: request sent, expect a webhook notification"
<Response [200]>
{'connection_id': '7a953984-42a6-4e29-ab76-cb31ac9573e4',
 'created_at': '2020-11-03 09:03:16.658027Z',
 'services': [{'consent_schema': {'data_dri': 'zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': '50f52ff5-c518-42b3-8028-0ef5e1cfd513',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}},
              {'consent_schema': {'data_dri': 'zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
         

In [37]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
  "payload": "abckonoin",
  "service": service_list['services'][0],
})
apply = json.loads(apply.text)
print(apply)

JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [53]:
time.sleep(1)
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue-self', json={
    "exchange_id": apply['exchange_id'],
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
[{'author': 'other',
  'connection_id': '1eb61440-f87c-47fb-938c-f5963df6295d',
  'consent_schema': '{"oca_schema_namespace": "consent", "oca_schema_dri": '
                    '"fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", '
                    '"data_dri": '
                    '"zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR"}',
  'created_at': '2020-10-26 12:41:42.789958Z',
  'exchange_id': '0c07b4ff-1a4b-4bf5-9c33-739ef203d59f',
  'issue_id': '0cf66f056d2cb2d85ae56a4fa54b2968e96eea8b478310546f6a696ee711a02e',
  'label': 'consent_test',
  'payload': None,
  'service_consent_match_id': 'a5c4ab1e-7a12-4916-b4e1-7c5d3294f666',
  'service_id': '07cda35c-4421-4a9a-b775-3f968f0466c3',
  'service_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string"}',
  'state': 'pending',
  'updated_at': '2020-10-26 12:41:42.789958Z'}]


In [55]:
issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
{'credential_exchange_record': {'auto_issue': True,
                                'auto_offer': False,
                                'auto_remove': False,
                                'connection_id': '1eb61440-f87c-47fb-938c-f5963df6295d',
                                'created_at': '2020-10-26 12:42:09.537333Z',
                                'credential_definition_id': '3LVMbg6wsELB3hycZ3ykEt:3:CL:16:Services',
                                'credential_exchange_id': 'cf9e83e2-c5c0-4681-a873-8f0993298584',
                                'credential_offer': {'cred_def_id': '3LVMbg6wsELB3hycZ3ykEt:3:CL:16:Services',
                                                     'key_correctness_proof': {'c': '100076842902840564216156485813849060267645390359882089417497726802921127736947',
                                                                               'xr_cap': [['service_consent_match_id',
                                                             

In [57]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': [{'attrs': {'data_dri': 'zQmSnRDrp3sNzsB194RaKwqKWmFS7mbT8oiF7qMWUCoNGgQ',
                        'oca_schema_dri': 'string',
                        'oca_schema_namespace': 'string',
                        'service_consent_match_id': 'a5c4ab1e-7a12-4916-b4e1-7c5d3294f666'},
              'cred_def_id': '3LVMbg6wsELB3hycZ3ykEt:3:CL:16:Services',
              'cred_rev_id': None,
              'referent': '6cb50e53-2ad8-4f49-984f-9076ee6f8a25',
              'rev_reg_id': None,
              'schema_id': '3LVMbg6wsELB3hycZ3ykEt:2:consent_test:1.0'}]}


In [ ]:
proof_schema = {
  "proof_request": {
    "version": "1.0",
    "name": "Proof request",
    "requested_attributes": {
      "data_dri": {
        "non_revoked": {
          "from_epoch": 1500000000,
          "to_epoch": 1700758817
        },
        "restrictions": [
          {
            "cred_def_id": apply['credential_definition_id'],
          }
        ],
        "name": "data_dri"
      },
    },
    "requested_predicates": {
    }
  },
  "connection_id": connection[1],
#   "trace": True,
  "comment": "string",
  "nonce": "1234567890"
}
proof_json = json.dumps(proof_schema)

proof = requests.post('http://localhost:8151/present-proof/send-request', json=proof_schema)
print(proof.text)

In [359]:
proof = requests.get('http://localhost:8150/present-proof/records')
proof_requests = json.loads(proof.text)['results']
proof_request = proof_requests[-4]
pp.pprint(proof_request)

{'connection_id': '994ebd0f-14c2-428c-be53-e730bc3ca7a7',
 'created_at': '2020-09-22 11:58:22.954779Z',
 'initiator': 'external',
 'presentation_exchange_id': '964c067c-46fb-4a4f-b30d-253029bf6d53',
 'presentation_request': {'name': 'Proof request',
                          'nonce': '236523038637111089936187',
                          'requested_attributes': {'data_dri': {'name': 'data_dri',
                                                                'non_revoked': {'from_epoch': 1500000000,
                                                                                'to_epoch': 1700758817},
                                                                'restrictions': [{'cred_def_id': 'WWZtMuY1V9hvFF1ezey2EM:3:CL:1175:Services'}]}},
                          'requested_predicates': {},
                          'version': '1.0'},
 'role': 'prover',
 'state': 'request_received',
 'thread_id': 'aab467d6-30db-442a-a23e-354f77100467',
 'trace': False,
 'updated_at': '2020-09-22 

In [360]:
url = 'http://localhost:8150/present-proof/records/' + \
       proof_request['presentation_exchange_id'] + \
       '/send-presentation'

presentation_schema = {
  "requested_attributes": {
    "data_dri": {
      "cred_id": credentials['results'][0]['referent'],
        "revealed": True
    },
  },
   "requested_predicates": {
  },
  "self_attested_attributes": {
  }
}
print(presentation_schema)
present = requests.post(url, json=presentation_schema)

print(present.text)

{'requested_attributes': {'data_dri': {'cred_id': '657ec1be-2d72-41ba-a1c2-725082d1c759', 'revealed': True}}, 'requested_predicates': {}, 'self_attested_attributes': {}}
{"initiator": "external", "presentation_exchange_id": "964c067c-46fb-4a4f-b30d-253029bf6d53", "created_at": "2020-09-22 11:58:22.954779Z", "trace": false, "state": "presentation_sent", "connection_id": "994ebd0f-14c2-428c-be53-e730bc3ca7a7", "presentation_request": {"version": "1.0", "name": "Proof request", "requested_attributes": {"data_dri": {"non_revoked": {"from_epoch": 1500000000, "to_epoch": 1700758817}, "restrictions": [{"cred_def_id": "WWZtMuY1V9hvFF1ezey2EM:3:CL:1175:Services"}], "name": "data_dri"}}, "requested_predicates": {}, "nonce": "236523038637111089936187"}, "thread_id": "aab467d6-30db-442a-a23e-354f77100467", "updated_at": "2020-09-22 11:58:42.143421Z", "presentation": {"proof": {"proofs": [{"primary_proof": {"eq_proof": {"revealed_attrs": {"data_dri": "46796680751272472783988362602137825638383598079

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)